# Fairness example with XAIoGraphs


* This notebook shows an example of using the "Fairness" module of the XAIoGraphs library.


* A dataset with tabulated data is needed to solve a classification problem with some ML model. This dataset has to have the following columns:

    + 'N' feature columns
    + 'S' columns with sensitive features $s \ \epsilon \ N$. The values of these sensitive features have to be discretized.
    + 'Real target' (y_true)
    + 'Predictions' (y_predict), made by ML model.
    
    
* From this dataset (Pandas DataFrame) XAIoGraphs will explain how fair or unfair the ML model is when making predictions (classifications) over sensitive features. These explanations are based on the 3 equity criteria:

    + **Independence:** We say that the random variables (Y, A) satisfy independence if the sensitive feature 'A' are statistically independent of the prediction 'Y'. 
    $$independence = P(Y=y∣A=a)$$
    
    + **Separation:** We say the random variables (Y, A, T) satisfy separation if the sensitive characteristics 'A' are statistically independent of the prediction 'Y' given the target value 'T'. We define the score as the difference (in absolute value) of the probabilities:
    $$separation = P(Y=y∣T=t,A=a)$$
    
    + **Sufficiency:** We say the random variables (Y,A,T) satisfy sufficiency if the sensitive characteristics 'A' are statistically independent of the target value 'T' given the prediction 'Y'. We define the score as the difference (in absolute value) of the probabilities:
    $$sufficiency = P(T=t∣Y=y,A=a)$$


* To carry out this example we are going to use the dataset "Body Performace" (available in: https://www.kaggle.com/datasets/kukuroo3/body-performance-data) that contains the grade of physical performance (A, B, C or D) based on age and some exercise performance data.


* We will create a predictive model that classifies each element in one of the 4 classes and we will study its equity based on the gender and age variables (discretized age ranges).

<hr>


## 1.- Classification model and predictions

### 1.1.- Read Dataset

* Once the dataset has been downloaded, it is read in csv as a pandas DataFrame.

* We replace the name of the "class" column with "y_true".

In [1]:
import pandas as pd

# Read Dataset
df = pd.read_csv('../../datasets/bodyPerformance.csv').rename({'class': 'y_true'}, axis=1)
print('Dataset Shape {}'.format(df.shape))
df.sample(3)

Dataset Shape (13393, 12)


,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,y_true
3215,26.0,F,157.6,50.6,30.4,65.0,103.0,26.5,15.3,50.0,165.0,B
5184,35.0,M,184.5,98.4,28.8,73.0,155.0,40.8,-12.5,51.0,167.0,D
6643,21.0,M,173.4,61.8,16.1,82.0,128.0,40.3,29.2,57.0,278.0,C


### 1.2.- Discretization "age" Feature

* We create a new feature with the discretized age ranges.

* Although it doesn't make sense, we keep the two features to show the "high correlation of variables" functionality of XAIoGraphs.

In [2]:
df['age_range'] = df['age'].apply(lambda x: '20-29' if (x >= 20 and x < 30)
                                  else ('30-39' if (x >= 30 and x < 40)
                                        else ('40-49' if (x >= 40 and x < 50)
                                              else ('50-59' if (x >= 50 and x < 60)
                                                    else '60-inf'))))

# Set 'age_range' columns in first position
df = df[df.columns.tolist()[-1:] +  df.columns.tolist()[:-1]]
df.sample(3)

,age_range,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,y_true
1331,30-39,35.0,M,167.7,68.8,14.8,82.0,125.0,50.5,22.7,62.0,256.0,A
3902,20-29,25.0,M,170.1,59.6,21.2,86.0,156.0,39.6,5.0,44.0,205.0,D
3889,20-29,28.0,M,171.5,69.6,24.2,99.0,153.0,36.2,6.6,43.0,220.0,C


### 1.3.- Label encoder


* For the ML algorithm, we code the discrete variables to numeric.

In [3]:
from sklearn.preprocessing import LabelEncoder

lb_gen = LabelEncoder()
lb_age_range = LabelEncoder()
lb_y = LabelEncoder()
df['gender'] = lb_gen.fit_transform(df['gender'])
df['age_range'] = lb_age_range.fit_transform(df['age_range'])
df['y_true'] = lb_y.fit_transform(df['y_true'])
df.sample(3)

,age_range,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,y_true
12134,3,55.0,0,156.8,52.6,27.6,90.0,149.0,21.5,24.35,29.0,122.0,1
5048,0,25.0,1,177.8,83.5,23.1,95.0,152.0,43.6,4.50,32.0,194.0,3
10938,0,25.0,0,156.0,46.8,20.2,77.0,128.0,22.1,21.90,42.0,179.0,0


### 1.4.- Train-Test split


In [4]:
from sklearn.model_selection import train_test_split

X = df[df.columns.drop('y_true')]
y = df['y_true']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


### 1.5.- Train Model & Performance measurement

* To make the Fairness example more didactic, we generate a model that is not very accurate.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Random Forest
model = RandomForestClassifier(n_estimators=20, bootstrap=True, criterion='gini', max_depth=10, random_state=123)
model.fit(X_train, y_train)

# Evaluation Metrics
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('Train Accuracy: {}'.format(accuracy_score(y_true=y_train, y_pred=y_train_pred)))
print('Test Accuracy: {}'.format(accuracy_score(y_true=y_test, y_pred=y_test_pred)))
print('Test Metrics:\n{}'.format(classification_report(y_true=y_test, y_pred=y_test_pred)))


Train Accuracy: 0.8424533333333334
Test Accuracy: 0.7155301144848183
Test Metrics:
              precision    recall  f1-score   support

           0       0.71      0.86      0.78      1024
           1       0.58      0.58      0.58       987
           2       0.71      0.63      0.67      1039
           3       0.89      0.79      0.83       968

    accuracy                           0.72      4018
   macro avg       0.72      0.72      0.72      4018
weighted avg       0.72      0.72      0.71      4018



### 1.6.- Predict all Dataset

* With the model trained, we calculate the predictions of all the elements in a new column (y_predict)

In [6]:
from copy import deepcopy
x_cols = df.columns.drop('y_true').tolist()

# Calculate predictions
df['y_predict'] = model.predict(df[x_cols])
df.sample(3)

,age_range,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,y_true,y_predict
1360,2,47.0,1,176.4,95.9,18.90000,95.0,146.0,53.4,-0.9,44.0,202.0,3,3
1253,0,22.0,0,168.2,56.2,19.45304,94.0,153.0,29.7,32.7,37.0,193.0,0,0
7568,4,63.0,0,160.3,54.5,29.60000,79.0,139.0,24.0,25.7,20.0,108.0,1,1


### 1.7.- Undo label encoder features

* XAIoGraphs works with discretized features (for those variables you have to work with); therefore, we do an inverse transformation of the values of the features.

In [7]:
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

df['age_range'] = lb_age_range.inverse_transform(df['age_range'])
df['gender'] = lb_gen.inverse_transform(df['gender'])
df['y_true'] = lb_y.inverse_transform(df['y_true'])
df['y_predict'] = lb_y.inverse_transform(df['y_predict'])
df.sample(3)

,age_range,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,y_true,y_predict
2126,60-inf,60.0,M,168.0,59.00,21.1,84.0,158.0,37.2,15.9,37.0,191.0,A,A
11978,60-inf,61.0,M,176.4,70.18,18.3,85.0,138.0,38.0,5.6,26.0,152.0,C,C
37,30-39,31.0,M,177.5,79.50,23.0,90.0,148.0,51.2,18.4,62.0,208.0,A,A


<hr>

# Fairness with XAIoGraphs


* We are going to study the sensitive features (discretized) 'gender' & 'age_range'.

* Once the Fairness module has been imported, we create an object of the Fairness class and call the "fit_fairness()" method with the following parameters:

    + **df:** Pandas DataFrame, with dataset -> Features, Sensitive Features, y_true, y_predict
    + **sensitive_cols:** List, with name of sensitive features to studied
    + **target_col:** str, with name of real target column
    + **predict_col:** str, with name of prediction column
    
* Upon execution, it will perform all Fairness calculations and they will be accessible by the "Property" functions of the class.

* It will also generate a series of csv files (in the "xaiographs_web_files" folder that is passed as a parameter in the class constructor) with the necessary information to display the data in the web interface.

In [8]:
from xaiographs.fairness import Fairness

f = Fairness(destination_path='./xaiographs_web_files', verbose=1)

f.fit_fairness(df=df, 
               sensitive_cols=['age_range', 'gender'], 
               target_col='y_true', 
               predict_col='y_predict')


Checking "broad jump_cm" column: 100%|███████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 364.61it/s]


Highly correlated variables above the 0.9 Threshold
  feature_1  feature_2  correlation_value  is_correlation_sensible
0       age  age_range           0.979564                     True


Processing: sensitive_col=age_range, sensitive_value=60-inf, target_label=A : 100%|█████████████████████████████████| 5/5 [00:00<00:00, 15.47it/s]
Processing: sensitive_col=age_range, sensitive_value=60-inf, target_label=B : 100%|█████████████████████████████████| 5/5 [00:00<00:00, 15.97it/s]
Processing: sensitive_col=age_range, sensitive_value=60-inf, target_label=D : 100%|█████████████████████████████████| 5/5 [00:00<00:00, 16.02it/s]
Processing: sensitive_col=age_range, sensitive_value=60-inf, target_label=C : 100%|█████████████████████████████████| 5/5 [00:00<00:00, 15.72it/s]
Processing: sensitive_col=gender, sensitive_value=M, target_label=A :   0%|                                                 | 0/2 [00:00<?, ?it/s]
Processing: sensitive_col=gender, sensitive_value=M, target_label=B :   0%|                                                 | 0/2 [00:00<?, ?it/s]
Processing: sensitive_col=gender, sensitive_value=M, target_label=D :   0%|                                           

In [9]:
f.fairness_info

,sensitive_feature,sensitive_value,is_binary_sensitive_feature,target_label,independence_score,independence_category,independence_score_weight,separation_score,separation_category,separation_score_weight,sufficiency_score,sufficiency_category,sufficiency_score_weight
0,age_range,20-29,False,A,0.091699,C,0.155454,0.102672,C,0.155454,0.013776,A+,0.118271
1,age_range,30-39,False,A,0.051340,B,0.069290,0.007547,A+,0.069290,0.008601,A+,0.055477
2,age_range,40-49,False,A,0.052768,B,0.036512,0.010660,A+,0.036512,0.033104,A,0.028821
3,age_range,50-59,False,A,0.119848,C,0.026432,0.103097,C,0.026432,0.001776,A+,0.023968
4,age_range,60-inf,False,A,0.120510,C,0.019786,0.198523,D,0.019786,0.137346,C,0.023445
5,age_range,20-29,False,B,0.061090,B,0.092063,0.064757,B,0.092063,0.084332,C,0.105279
6,age_range,30-39,False,B,0.008397,A+,0.050549,0.036001,A,0.050549,0.011037,A+,0.052490
7,age_range,40-49,False,B,0.024911,A,0.031509,0.040302,A,0.031509,0.027575,A,0.032330
8,age_range,50-59,False,B,0.053249,B,0.038229,0.079066,B,0.038229,0.069346,B,0.032330
9,age_range,60-inf,False,B,0.118553,C,0.035242,0.177754,D,0.035242,0.047412,A,0.027477
